In [7]:
gt= pd.read_csv('Table dataset.csv')
gt.head()

,Prompt,Winner
0,"Oxides weight percentage content in EAF slag, ...","Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, S..."
1,Properties of cement and fly ashes used \n [['...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
2,Chemical composition of SFS. \n [['Constituent...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
3,"Chemical compositions of cement, silica fume a...","Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
4,"Cement composition oxide wt.% \n [['SiO2', 'Al...","Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."


In [25]:
dataset_c = []
for _, (Table, Winner) in gt.iterrows():
    Prompt = f"""CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.
    Handling Missing Data: Use 'null' for any missing data in these columns.
    Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.

    Unstructured HTML table: {Table}"""


    dataset_c.append({"Prompt": Prompt, "Winner": str(Winner).replace("NaN","null")})
    
len(dataset_c)

200

In [26]:
gt.loc[1].Winner

'Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,LOI\nPC,20.2,5.9,3.3,63.3,1.8,2.9,NaN,NaN,0.8,0.5,0.8\nHL1,43.3,25.1,4.9,20.3,0.8,2.5,NaN,NaN,1.3,0.2,1.19\nHL2,47.1,17.4,8.3,14.0,1.9,4.6,NaN,NaN,1.8,2.4,1.7\nLL1,58.6,21.9,9.3,4.4,1.4,0.4,NaN,NaN,1.8,0.3,1.4\nLL2,59.4,25.8,5.8,2.0,0.6,0.1,NaN,NaN,3.8,0.4,1.6'

In [27]:
# Reformatting for fine-tuning
fine_tuning_dataset = []
for item in dataset_c:
    fine_tuning_item = {
        "messages": [
            {"role": "system", "content": "You are an assistant trained to parse unstructured HTML tables into CSV format."},
            {"role": "user", "content": json.dumps(item["Prompt"])},
            {"role": "assistant", "content": json.dumps(item["Winner"])}
        ]
    }
    fine_tuning_dataset.append(fine_tuning_item)
    


In [28]:
len(fine_tuning_dataset)

200

In [29]:
fine_tuning_dataset[1]

{'messages': [{'role': 'system',
   'content': 'You are an assistant trained to parse unstructured HTML tables into CSV format.'},
  {'role': 'user',
   'content': '"CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.\\n    Handling Missing Data: Use \'null\' for any missing data in these columns.\\n    Non-relevant Data: If the table\'s content is not related to the specified chemicals, respond with \'NR\' only.\\n\\n    Unstructured HTML table: Properties of cement and fly ashes used \\n [[\'Chemical composition\', \'Chemical composition\', \'Chemical composition\', \'Chemical composition\', \'Chemical composition\', \'\'], [\'Oxide (%)\', \'PC\', \'HL1\', \'HL2\', \'LL1\', \'LL2\'], [\'CaO\', \'63.3\', \'20.3\', \'14.0\', \'4.4\', \'2.0\'], [\'SiO2\', \'20.2\', \'43.3\', \'47.1\', \'58.6\', \'59.4\'], [\'Al2O3\', \'5.9\', \'25.1\', \'17.4\', \'21.9\', \'25.8\'], [\'Fe2O3\', \'3.3\', \'4.9\', \'8.3\

In [30]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(fine_tuning_dataset, test_size=0.1)

In [31]:

with open('fine_tuning_TE_CC.jsonl', 'w') as file:
    for item in fine_tuning_dataset:
        json.dump(item, file)
        file.write('\n')
        

with open('fine_tuning_TRAIN_TE_CC.jsonl', 'w') as file:
    for item in train_data:
        json.dump(item, file)
        file.write('\n')
        

with open('fine_tuning_TEST_TE_CC.jsonl', 'w') as file:
    for item in test_data:
        json.dump(item, file)
        file.write('\n')

In [32]:
len(fine_tuning_dataset)

200

In [ ]:
api_key=api_key

In [33]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer ${api_key}" \
  -F "purpose=fine-tune" \
  -F "file=@fine_tuning_TE_CC.jsonl" 

{
  "object": "file",
  "id": "file-R9fcrmV8QimJm5bRkdnWMeyn",
  "purpose": "fine-tune",
  "filename": "fine_tuning_TE_CC.jsonl",
  "bytes": 260979,
  "created_at": 1705548780,
  "status": "processed",
  "status_details": null
}


In [34]:
!curl https://api.openai.com/v1/files \
  -H r"Authorization: Bearer ${api_key}" \
  -F "purpose=fine-tune" \
  -F "file=@fine_tuning_TRAIN_TE_CC.jsonl" 

{
  "object": "file",
  "id": "file-1beLBpaYIuSKWNXVLXs3gM42",
  "purpose": "fine-tune",
  "filename": "fine_tuning_TRAIN_TE_CC.jsonl",
  "bytes": 238224,
  "created_at": 1705548782,
  "status": "processed",
  "status_details": null
}


In [35]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer ${api_key} \
  -F "purpose=fine-tune" \
  -F "file=@fine_tuning_TEST_TE_CC.jsonl" 

{
  "object": "file",
  "id": "file-YtcQNMpcbEOsGbbGAA8EtkyX",
  "purpose": "fine-tune",
  "filename": "fine_tuning_TEST_TE_CC.jsonl",
  "bytes": 22755,
  "created_at": 1705548782,
  "status": "processed",
  "status_details": null
}


In [36]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

client.fine_tuning.jobs.create(
  training_file="file-q0vjmL1s9EIrud1SsSO1vLT7", 
  validation_file="file-XMf29I1HR1H2O7FrQeO4WzmN", 
  model="gpt-3.5-turbo-1106", 
  hyperparameters={
    "n_epochs":"auto"
  }
)

FineTuningJob(id='ftjob-wgyS1iM7UHyiSxPS6epXJ4Df', created_at=1705548783, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-CGIKMf8W0RLVflPNXCgE6SzT', result_files=[], status='validating_files', trained_tokens=None, training_file='file-q0vjmL1s9EIrud1SsSO1vLT7', validation_file='file-XMf29I1HR1H2O7FrQeO4WzmN')

In [37]:
#client.fine_tuning.jobs.cancel("ftjob-dKkESY7D9zgJxlIczb1bR82E")


In [38]:
!curl https://api.openai.com/v1/fine_tuning/jobs/ftjob-Y5A5vvGYACRcuHtRf4tWLRW1 \
  -H "Authorization: Bearer ${api_key}"

{
  "object": "fine_tuning.job",
  "id": "ftjob-Y5A5vvGYACRcuHtRf4tWLRW1",
  "model": "gpt-3.5-turbo-1106",
  "created_at": 1704920850,
  "finished_at": 1704923268,
  "fine_tuned_model": "ft:gpt-3.5-turbo-1106:personal::8faiLnzJ",
  "organization_id": "org-CGIKMf8W0RLVflPNXCgE6SzT",
  "result_files": [
    "file-iZZ17sdaLGNJpKUemqmO0LQv"
  ],
  "status": "succeeded",
  "validation_file": "file-XMf29I1HR1H2O7FrQeO4WzmN",
  "training_file": "file-q0vjmL1s9EIrud1SsSO1vLT7",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 269460,
  "error": null
}

In [39]:
!curl https://api.openai.com/v1/files/file-iZZ17sdaLGNJpKUemqmO0LQv/content \
  -H "Authorization: Bearer ${api_key}" > file.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10724    0 10724    0     0  17409      0 --:--:-- --:--:-- --:--:-- 17380


In [40]:
# Initialize embeddings and Chroma DB
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
persist_directory = 'data/chroma/etsminilm'

tvdb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [91]:
question = "major oxides or chemical compositions of materials including Al2O3, Fe2O3, CaO, MgO, So3, Tio2, Mno, K2o, Na2O, LOI, loss of ignition, ignition loss"
docs = tvdb.similarity_search(
    question,
    k=100000)
#del docs[:70000]

In [42]:
def save_responses(responses):
    # Implement the logic to save 'responses'
    # For example, saving to a file:
    filename = f'responses_some.csv'
    # Convert responses to DataFrame and save as CSV
    pd.DataFrame(responses, columns=['Response', 'Caption', 'DOI']).to_csv(filename, index=False)

In [43]:

from openai import OpenAI
client = OpenAI(api_key=api_key)



In [44]:
error_tables = [] 
responses = []

In [ ]:

for i, doc in enumerate(tqdm(docs)):

    try: 
        Table = doc.page_content  # The caption of the table
        caption = Table.split('/n')[0]
        doi = doc.metadata['doi']  # The DOI of the table

        AA = json.dumps(f"""CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.
        Handling Missing Data: Use 'null' for any missing data in these columns.
        Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.

        Unstructured HTML table: {Table}""")

        # Check if this caption and DOI already exist in the existing DataFrame
        response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:personal::8faiLnzJ",messages=[
        {"role": "system", "content": "You are an assistant trained to parse unstructured HTML tables into CSV format."},
        {"role": "user", "content": f"""CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.
        Handling Missing Data: Use 'null' for any missing data in these columns.
        Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.

        Unstructured HTML table: {Table}"""}],
        temperature=0.0,
        max_tokens=800,
        top_p=1,)

        RR=response.choices[0].message.content
        responses.append((RR, caption, doi))

        # Check if it's time to save the responses
        if (i + 1) % 300 == 0:
            save_responses(responses)


    except Exception as e:
        error_tables.append((Table, e))  # Append the table and the error for later review

In [61]:
keywords = {"al2o3", "sio2", "fe2o3", "cao", "mgo", "so3", "tio2", "mno", "k2o", "na2o", "loi", "loss of ignition", "ignition loss"}

for i, doc in enumerate(tqdm(docs)):
    if any(keyword in str(doc).lower() for keyword in keywords):
        try: 
            Table = doc.page_content  # The caption of the table
            caption = Table.split('/n')[0]
            doi = doc.metadata['doi']  # The DOI of the table

            AA = json.dumps(f"""CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.
            Handling Missing Data: Use 'null' for any missing data in these columns.
            Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.

            Unstructured HTML table: {Table}""")

            # Check if this caption and DOI already exist in the existing DataFrame
            response = client.chat.completions.create(
            model="ft:gpt-3.5-turbo-1106:personal::8faiLnzJ",messages=[
            {"role": "system", "content": "You are an assistant trained to parse unstructured HTML tables into CSV format."},
            {"role": "user", "content": f"""CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.
            Handling Missing Data: Use 'null' for any missing data in these columns.
            Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.

            Unstructured HTML table: {Table}"""}],
            temperature=0.0,
            max_tokens=800,
            top_p=1,)

            RR=response.choices[0].message.content
            responses.append((RR, caption, doi))

            # Check if it's time to save the responses
            if (i + 1) % 300 == 0:
                save_responses(responses)


        except Exception as e:
            error_tables.append((Table, e))  # Append the table and the error for later review

100%|██████████| 30001/30001 [01:09<00:00, 434.79it/s] 


In [62]:
save_responses(responses)

In [63]:
import pandas as pd
import io

def is_header(row, num_columns):
    """
    Check if a row can be considered as a header based on its format.
    """
    columns = row.split(",")
    if len(columns) != num_columns and column[0]=='Material Name':
        return False
    
def parse_response_to_df(response, expected_columns, caption, doi):
    # Clean the response string
    cleaned_response = str(response).replace("null","NaN")
    cleaned_response = ''.join(map(str, cleaned_response))
    cleaned_response = cleaned_response.strip('[').strip(']').strip('""').strip('''''').strip('\'"').replace('\\n', '\n').replace('\\', '')
    # Convert expected columns to lower case
    expected_columns_lower = [col.lower() for col in expected_columns]
    
    if cleaned_response == ['NR']: return pd.DataFrame(columns=expected_columns) , []
    
    # Process each row
    conf_tables = []
    
    # Initialize a new DataFrame with lowercased expected columns
    new_df = pd.DataFrame(columns=expected_columns_lower)

    try:
        # Convert the cleaned response string to a DataFrame
        df = pd.read_csv(io.StringIO(cleaned_response))
        # Convert all column names in df to lower case for comparison
        df.columns = [col.lower() for col in df.columns]
        df.columns = df.columns.str.strip()

        # Copy data from original DataFrame or fill with NaN
        for exp_col in expected_columns_lower:
            if exp_col in df.columns:
                new_df[exp_col] = df[exp_col]
            else:
                new_df[exp_col] = pd.NA

        return new_df, conf_tables

    except Exception as e:
        print(e)
        print(cleaned_response)

        return pd.DataFrame(columns=expected_columns_lower), conf_tables


In [64]:



# Expected columns
expected_columns = ['Material Name', 'SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'SO3', 'TiO2', 'MnO', 'K2O', 'Na2O', 'LOI']

# Create a DataFrame for each response
dfs = []
conf_tables = []
for response_data, caption, doi in responses:
    df , conf_table = parse_response_to_df(response_data, expected_columns, caption, doi)

    # Add 'caption' and 'doi' columns to the DataFrame
    df['caption'] = caption
    df['doi'] = doi
    
    dfs.append(df)
    conf_tables.append(conf_table)

# Concatenate all DataFrames into one
final_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(columns=expected_columns + ['caption', 'doi'])

Error tokenizing data. C error: Expected 12 fields in line 3, saw 13

Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,LOI
Normal cement,20.7,4.96,3.33,65.2,0.57,2.49,NaN,NaN,0.32,0.10,NaN
Cement with glass addition, second test,21.3,5.37,3.37,65.1,0.61,2.47,NaN,NaN,0.31,0.20,NaN
Error tokenizing data. C error: Expected 12 fields in line 9, saw 13

Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,LOI
Glass 1,80.5,2.2,NaN,NaN,NaN,NaN,NaN,NaN,0.4,3.8,NaN
Glass 2,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN
Glass 3,77.9,0.8,NaN,0.8,NaN,NaN,NaN,NaN,2.3,4.1,NaN
Glass 4,71-73.5,NaN,0.01,5-6,3.5-4.5,NaN,NaN,NaN,0-1.5,14-17,NaN
Glass 5,61.5,NaN,0.8,NaN,NaN,NaN,NaN,NaN,5.3,8.8,NaN
Glass 6,72.68,NaN,NaN,12.76,0.26,NaN,NaN,NaN,NaN,13.25,NaN
Glass 7,63.0,NaN,NaN,0.3,0.2,NaN,NaN,NaN,6.0,7.6,NaN
Glass 8,72.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN
Glass 9,57.3,3.2,NaN,4.7,8.7,NaN,NaN,NaN,0.5,1.9,NaN
Glass 10,75.3,6.2,NaN,1.1,NaN,NaN,NaN,NaN,0.8,5.7,NaN
Glass 11,68.0,3.0,

In [65]:
# Save the DataFrame as a CSV file
final_df.to_csv('all_materials_extracted_fgpt.csv', index=False)
